In [ ]:
"""
Colab Notebook Set Up

Use this cell to upload your kaggle.json file as well as the `download_data.sh`
and `preprocess.py` scripts.
"""

from google.colab import files
import os

# Upload files
kaggle = files.upload()
data_script = files.upload()
preprocess_script = files.upload()
config = files.upload()

# Verify uploads
for file in ["kaggle.json", "download_data.sh", "preprocess.py", "oct.yaml"]:
    assert file in os.listdir(), f"Make sure you upload the {file} file"

# Shell commands (files)
!mkdir -p ~/.kaggle/ data/ models/ config/ scripts/ net/
!mv kaggle.json ~/.kaggle/
!mv download_data.sh preprocess.py scripts/
!mv oct.yaml config/
!chmod 600 ~/.kaggle/kaggle.json
!chmod +x scripts/download_data.sh scripts/preprocess.py
!sed -i -e 's/\r$//' scripts/download_data.sh
!pip install -q kaggle pretrainedmodels rich
!touch net/__init__.py net/train.py net/utils.py

# Run shell commands
!scripts/download_data.sh

In [ ]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

%tensorboard --logdir runs

## Finetuning InceptionV3 for Retinal OCT Images

### Context

- Retinal Optical Coherence Tomography (OCT) is an imaging technique used to
  capture high-res cross sections of the retina
- ~84, 495 OCT Images in total

### Content

- Images in JPEG format with 3 channels, i.e., RGB
- 4 categories: CNV, DME, DRUSEN, NORMAL

### This Notebook

- Fine-tune InceptionV3 by training the last, linear layer on the new data
- During the forward pass, the images will be passed through the full
  network, but only the weights of the last layer will be updated based on
  the loss

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch import nn, optim
import pretrainedmodels as models
import torch

from datetime import datetime
from tqdm.auto import tqdm
from rich import print
from glob import glob

import sys; sys.path.append(".")
from net import train, utils

import numpy as np
import copy
import yaml
import os
import re


config = "config/oct.yaml"
with open(config, 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((config["input-height"], config["input-width"])),
    transforms.ToTensor(),
    transforms.Normalize(config["input-mean"], config["input-std"]),
])

# Define datasets
data = {
    "train": datasets.ImageFolder(config["train-dir"], transform),
    "test": datasets.ImageFolder(config["test-dir"], transform),
    "val": datasets.ImageFolder(config["val-dir"], transform),
}

# Define dataloaders
dataloaders = {
    "train": DataLoader(data["train"], batch_size=config["batch-size"], shuffle=True),
    "test": DataLoader(data["test"], batch_size=config["batch-size"], shuffle=True),
    "val": DataLoader(data["val"], batch_size=config["batch-size"], shuffle=False),
}

In [ ]:
class FineTuned(nn.Module):
    """Fine-tuned output layer for InceptionV3"""

    def __init__(self, config):
        super(FineTuned, self).__init__()

        self.model = models.__dict__[config["model-name"]](num_classes=1000, pretrained="imagenet")
        for param in self.model.parameters():
            param.requires_grad = False

        self.model.last_linear = nn.Linear(self.model.last_linear.in_features, config["num-classes"])

    def forward(self, x):
        x = self.model(x)
        return x
    
model = FineTuned(config)
model = model.to(train.device())

trainable = filter(lambda p: p.requires_grad, model.parameters())

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(trainable, lr=config["lr"])
# optimizer = optim.SGD(trainable, lr=config["lr"], momentum=config["momentum"])

In [ ]:
# Initialize training and logging
timestamp = datetime.now().strftime("%m%d_%H%M%S")
writer = SummaryWriter(f"logs/inception/oct-{timestamp}")
EPOCH = 0

In [ ]:
num_epochs = 10
EPOCH += num_epochs

model, history = train.train_model(
    model, dataloaders, criterion, optimizer, num_epochs, EPOCH, writer
)